# 2-class Classification

We want to assess the performance of a NN trained with a privacy engine based on SGD algorithm.
The dataset is public and references to its license could be found in the README.md in /data subdirectory of the repo.

In [1]:
# Let's call it the Ordinary stack
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white", color_codes=True)
%matplotlib inline

# These are the new cowboys in the town
# ignore it for now -> from petastorm import make_batch_reader
# ignore it for now -> from petastorm.pytorch import DataLoader
from pyspark.sql import SparkSession
import torch
import torch.nn as nn
import torch.nn.functional as F
import opacus

In [2]:
# This is the entry point of our spark app
spark = SparkSession \
        .builder \
        .appName("2-class classification") \
        .getOrCreate()

In [3]:
# Unfortunately there's no direct way to populate Spark's Dataframes from remote. So... here's a hack
url = "https://raw.githubusercontent.com/alessio-proietti/dp-sgd-notebook/main/data/bank-additional-full.csv"
pdf = pd.read_csv(url, delimiter=";")

# This is the real thing, our dataframe exposed as Spark's Dataframe
#df = spark.createDataFrame(pdf)
#df

In [4]:
df = spark.createDataFrame(pdf)

# !!!PLEASE DO NOT EXECUTE THIS CELL!!!

from pyspark.ml.feature import StringIndexer, OneHotEncoder
# We index the string category with numbers
stringIndexer = StringIndexer() \
    .setInputCols(["job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome","day_of_week"]) \
    .setOutputCols(["jobIndex", "maritalIndex", "educationIndex", "defaultIndex", "housingIndex", "loanIndex", "contactIndex", "monthIndex", "poutcomeIndex","day_of_weekIndex"])

stringModel = stringIndexer.fit(df)
df = stringModel.transform(df).drop("job", "marital", "education", "default", "housing", "loan", "contact", "month", "poutcome","day_of_week")

#to be deleted IS ONLY AN EXPERIMENT
df = df.drop("emp.var.rate", "cons.price.idx", "cons.conf.idx", "nr.employed", "y")
df.toPandas().head()

,age,duration,campaign,pdays,previous,euribor3m,defaultIndex,loanIndex,day_of_weekIndex,monthIndex,poutcomeIndex,educationIndex,contactIndex,maritalIndex,jobIndex,housingIndex
0,56,261,1,999,0,4.857,0.0,0.0,1.0,0.0,0.0,4.0,1.0,0.0,8.0,1.0
1,57,149,1,999,0,4.857,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0
2,37,226,1,999,0,4.857,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0
3,40,151,1,999,0,4.857,0.0,0.0,1.0,0.0,0.0,5.0,1.0,0.0,0.0,1.0
4,56,307,1,999,0,4.857,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,3.0,1.0


In [5]:
# The number of instances is quite large, we could attempt to have a tripartition of the dataset
train, validation, test = df.randomSplit([3.0, 1.0, 1.0], 24)

In [6]:
train_array = np.array(train.collect())
train_array
train_tensor = torch.from_numpy(train_array)
train_tensor[0]

tensor([ 20.0000, 217.0000,   2.0000, 999.0000,   0.0000,   4.8550,   0.0000,
          0.0000,   4.0000,   0.0000,   0.0000,   1.0000,   1.0000,   1.0000,
          6.0000,   1.0000], dtype=torch.float64)

In [7]:
spark.stop()